# Create crossval ext tables

This notebook is dedicated to create the cv table to all tunings produced during 2020 for tracking purpose.


In [1]:
from core import crossval_table, get_color_fader
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_curve
import tensorflow as tf
from Gaugi import mkdir_p
from copy import copy
from pprint import pprint
import numpy as np
import pandas
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

from tensorflow.keras.models import Model, model_from_json
from Gaugi import load as gload

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence

Applying ATLAS style settings...

Applying ATLAS style settings...
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...


In [2]:
mkdir_p('output/crossval')
#mkdir_p('output/training')

In [3]:
def create_op_dict(op, decoration='reference'):
    
    d = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              
              # op
              'pd_ref'    : decoration+"/"+op+"/pd_ref#0",
              'fa_ref'    : decoration+"/"+op+"/fa_ref#0",
              'sp_ref'    : decoration+"/"+op+"/sp_ref",
              'pd_val'    : decoration+"/"+op+"/pd_val#0",
              'fa_val'    : decoration+"/"+op+"/fa_val#0",
              'sp_val'    : decoration+"/"+op+"/sp_val",
              'pd_op'     : decoration+"/"+op+"/pd_op#0",
              'fa_op'     : decoration+"/"+op+"/fa_op#0",
              'sp_op'     : decoration+"/"+op+"/sp_op",

              # Counts
              'pd_ref_passed'    : decoration+"/"+op+"/pd_ref#1",
              'fa_ref_passed'    : decoration+"/"+op+"/fa_ref#1",
              'pd_ref_total'     : decoration+"/"+op+"/pd_ref#2",
              'fa_ref_total'     : decoration+"/"+op+"/fa_ref#2",
              'pd_val_passed'    : decoration+"/"+op+"/pd_val#1",
              'fa_val_passed'    : decoration+"/"+op+"/fa_val#1",
              'pd_val_total'     : decoration+"/"+op+"/pd_val#2",
              'fa_val_total'     : decoration+"/"+op+"/fa_val#2",
              'pd_op_passed'     : decoration+"/"+op+"/pd_op#1",
              'fa_op_passed'     : decoration+"/"+op+"/fa_op#1",
              'pd_op_total'      : decoration+"/"+op+"/pd_op#2",
              'fa_op_total'      : decoration+"/"+op+"/fa_op#2",
    })
    return d


op_names = ['tight', 'medium', 'loose', 'vloose']

tuned_info = collections.OrderedDict({})
for op in op_names:
    tuned_info[op] = create_op_dict(op, "reference")

In [4]:
etbins = [15, 20, 30, 40, 50,100, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:


In [5]:
cv  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [6]:
cv.from_csv('output/crossval/table_v12.csv')

In [7]:
best_inits = cv.filter_inits("max_sp_val")
best_inits = best_inits.loc[best_inits.model_idx==3]
best_sorts = cv.filter_sorts( best_inits , 'max_sp_op')

## Extended bins:

In [8]:
ext_table = best_inits.loc[best_inits.et_bin==4].reset_index(drop=True)
ext_table['et_bin'] = 5
best_inits = pandas.concat((best_inits, ext_table),axis=0,ignore_index=True)

In [9]:
basepath = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
datapath = basepath+'/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.h5'
paths = [ [datapath.format(ET=et_bin, ETA=eta_bin) for eta_bin in range(5)] for et_bin in range(5)]

In [10]:
# calculate all pd/fa from reference file
ref_path = '/home/jodafons/public/cern_data/new_files/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins/'
ref_path += 'new_references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins_et{ET}_eta{ETA}.ref.npz'
ref_paths = [[ ref_path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]
ref_values = [[ {} for eta in range(5)] for et in range(5)]

from saphyra.core import ReferenceReader
for et_bin in range(5):
    for eta_bin in range(5):
        for op_name in op_names:
            refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            _pd = refObj.getSgnPassed(op_name)/refObj.getSgnTotal(op_name)
            _fa = refObj.getBkgPassed(op_name)/refObj.getBkgTotal(op_name)
            ref_values[et_bin][eta_bin][op_name] = {'pd':_pd, 'fa':_fa}

2022-03-10 21:16:22.092427: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-03-10 21:16:22.092466: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: UNKNOWN ERROR (-1)
2022-03-10 21:16:22.092491: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (caloba51): /proc/driver/nvidia/version does not exist
2022-03-10 21:16:22.095404: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-10 21:16:22.188475: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2400000000 Hz
2022-03-10 21:16:22.197768: I tensorflow/compiler/xla/service/service.c

In [11]:
paths.append(paths[4])
ref_values.append(ref_values[4])

## Generators:

In [12]:
def getPatterns( path, etbin, etabin, kf, sort ):

    pidname = 'el_lhmedium'
    from kepler.pandas import load_hdf
    import numpy as np
    df = load_hdf(path)
    df = df.loc[ ((df[pidname]==True) & (df.target==1.0)) | ((df.target==0) & (df['el_lhvloose']==False) ) ]
    df = df.loc[ (df['trig_L2_cl_et']/1000  >= etbin[0]) & (df['trig_L2_cl_et']/1000  < etbin[1])]
    df = df.loc[ (abs(df['trig_L2_cl_eta'])  >= etabin[0]) & (abs(df['trig_L2_cl_eta'])  < etabin[1])]

  

    # for new training, we selected 1/2 of rings in each layer
    #pre-sample - 8 rings
    # EM1 - 64 rings
    # EM2 - 8 rings
    # EM3 - 8 rings
    # Had1 - 4 rings
    # Had2 - 4 rings
    # Had3 - 4 rings
    prefix = 'trig_L2_cl_ring_%i'

    # rings presmaple 
    presample = [prefix %iring for iring in range(8//2)]
    # EM1 list
    sum_rings = 8
    em1 = [prefix %iring for iring in range(sum_rings, sum_rings+(64//2))]
    # EM2 list
    sum_rings = 8+64
    em2 = [prefix %iring for iring in range(sum_rings, sum_rings+(8//2))]
    # EM3 list
    sum_rings = 8+64+8
    em3 = [prefix %iring for iring in range(sum_rings, sum_rings+(8//2))]
    # HAD1 list
    sum_rings = 8+64+8+8
    had1 = [prefix %iring for iring in range(sum_rings, sum_rings+(4//2))]
    # HAD2 list
    sum_rings = 8+64+8+8+4
    had2 = [prefix %iring for iring in range(sum_rings, sum_rings+(4//2))]
    # HAD3 list
    sum_rings = 8+64+8+8+4+4
    had3 = [prefix %iring for iring in range(sum_rings, sum_rings+(4//2))]
    col_names = presample+em1+em2+em3+had1+had2+had3
    rings = df[col_names].values.astype(np.float32)

    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    
    target = df['target'].values.astype(np.int16)
    data_rings = norm1(rings)
    
    # This is mandatory
    splits = [(train_index, val_index) for train_index, val_index in kf.split(data_rings,target)]
    # split for this sort
    x_train = [ data_rings[splits[sort][0]] ]
    x_val   = [ data_rings[splits[sort][1]] ]
    y_train = target [ splits[sort][0] ]
    y_val   = target [ splits[sort][1] ]
    
    return x_train, x_val, y_train, y_val


## Rerun references:

In [13]:
def sp_func(pd, fa):
    return np.sqrt(  np.sqrt(pd*(1-fa)) * (0.5*(pd+(1-fa)))  )

#
# Calculate sp, pd and fake given a reference
# 
def calculate( y_train, y_val , y_op, ref, pd,fa,sp,thresholds, 
               pd_val,fa_val,sp_val,thresholds_val, 
               pd_op,fa_op,sp_op,thresholds_op ):

    d = {}
    def closest( values , ref ):
        index = np.abs(values-ref)
        index = index.argmin()
        return values[index], index


    d['pd_ref'] = ref['pd']
    d['fa_ref'] = ref['fa']
    d['sp_ref'] = ref['sp']
    
    op_total = len(y_op[y_op==1])
    d['pd_ref_passed'] = int(ref['pd']*op_total)
    d['pd_ref_total']  = op_total
    op_total = len(y_op[y_op!=1])
    d['fa_ref_passed'] = int(ref['fa']*op_total)
    d['fa_ref_total']  = op_total

    _, index = closest( pd_val, d['pd_ref'] )
    val_total   = len(y_val[y_val==1])
    d['pd_val'] = pd_val[index]  
    d['pd_val_passed'] = int(val_total*float(pd_val[index]))
    d['pd_val_total']  = val_total
                   
    val_total   = len(y_val[y_val!=1])
    d['fa_val'] = fa_val[index]
    d['fa_val_passed'] = int(val_total*float(fa_val[index]))
    d['fa_val_total']  = val_total
    
    d['sp_val'] = sp_func(d['pd_val'], d['fa_val'])
 

    # Train + Validation
    _, index = closest( pd_op, d['pd_ref'] )
    op_total = len(y_op[y_op==1])
    d['pd_op'] = pd_op[index]
    d['pd_op_passed'] = int(op_total*float(pd_op[index]))
    d['pd_op_total'] = op_total
    
    op_total   = len(y_op[y_op!=1])
    d['fa_op'] = fa_op[index]
    d['fa_op_passed'] = int(op_total*float(fa_op[index]))
    d['fa_op_total']  = op_total
    
    d['sp_op'] = sp_func(d['pd_op'], d['fa_op'])

       
    return d

In [14]:
def rerun(table_df, paths, refs, data_generator, kf, etbins, etabins, et_bin, eta_bin):
        
    tf.config.run_functions_eagerly(False)

    bin_table = table_df.loc[ (table_df.et_bin==et_bin)&(table_df.eta_bin==eta_bin) ]
    
    et_bin_edges = (etbins[et_bin], etbins[et_bin+1])
    eta_bin_edges = (etabins[eta_bin], etabins[eta_bin+1])
    
    for sort in tqdm( bin_table.sort.unique() , desc='Running (%d,%d)'%(et_bin,eta_bin), ncols=70 , total=len(bin_table.sort.unique())):
        
        x_train, x_val, y_train, y_val = data_generator(paths[et_bin][eta_bin], et_bin_edges, eta_bin_edges, kf, sort )
        
        view = bin_table.loc[bin_table.sort==sort]
        
        file_name = view.file_name.values[0]
        tuned_idx = view.tuned_idx.values[0]
        model_idx = view.model_idx.values[0]
        tuned = gload(file_name)['tunedData'][tuned_idx]
        model = model_from_json( json.dumps(tuned['sequence'], separators=(',', ':')) ) #custom_objects={'RpLayer':RpLayer} )
        model.set_weights( tuned['weights'] )

        y_pred     = model.predict( x_train, batch_size = 1024, verbose=0 )
        y_pred_val = model.predict( x_val  , batch_size = 1024, verbose=0 )      
        
        # get vectors for operation mode (train+val)
        y_pred_op = np.concatenate( (y_pred, y_pred_val), axis=0)
        y_op = np.concatenate((y_train,y_val), axis=0)
            
        for idx, row in bin_table.loc[bin_table.sort==sort].iterrows():
            
            
            ref = {'pd':refs[et_bin][eta_bin][row.op_name]['pd'], 'fa':refs[et_bin][eta_bin][row.op_name]['fa']}
            ref['sp'] = sp_func(ref['pd'], ref['fa'])
            train_total = len(y_train)
            val_total = len(y_val)

            # Here, the threshold is variable and the best values will
            # be setted by the max sp value found in hte roc curve
            # Training
            fa, pd, thresholds = roc_curve(y_train, y_pred)
            sp = sp_func(pd, fa)

            # Validation
            fa_val, pd_val, thresholds_val = roc_curve(y_val, y_pred_val)
            sp_val = sp_func(pd_val, fa_val)
            knee = np.argmax(sp_val)
            table_df.at[idx, 'max_sp_val'] = sp_val[knee]
            #table_df.at[idx, 'max_pd_val'] = pd_val[knee]
            #table_df.at[idx, 'max_fa_val'] = fa_val[knee]


            # Operation
            fa_op, pd_op, thresholds_op = roc_curve(y_op, y_pred_op)
            sp_op = sp_func(pd_op, fa_op)
            knee = np.argmax(sp_op)
            table_df.at[idx, 'max_sp_op'] = sp_op[knee]
            #table_df.at[idx, 'max_pd_op'] = pd_op[knee]
            #table_df.at[idx, 'max_fa_op'] = fa_op[knee]

          
            d = calculate( y_train, y_val , y_op, ref, 
                               pd, fa, sp, thresholds, 
                               pd_val, fa_val, sp_val, thresholds_val, 
                               pd_op,fa_op,sp_op,thresholds_op )
          
            #print(d['pd_ref_total'])
         
            for key in d.keys():
                table_df.at[idx, key] = d[key]
            #print('sort = %d, pid = %s, pd_ref = %1.4f -> pd_op = %1.4f, fa_op = %1.4f, idx=%d, tot=%d'%(sort, row.op_name, ref['pd'], 
            #                                                                                     d['pd_op'], d['fa_op'],idx, d['pd_ref_total']) )

     
    return table_df


In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=512, shuffle=True)        
best_inits_ext = best_inits.copy()
etbins = [15, 20, 30, 40, 50, 100, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

for et_bin in [4,5]:
    for eta_bin in [0,1,2,3,4]:
        rerun(best_inits_ext, paths, ref_values, 
              getPatterns, kf, etbins, etabins, et_bin, eta_bin)

Running (5,4):  60%|███████████▍       | 6/10 [00:04<00:02,  1.34it/s]

In [ ]:
best_inits_ext.to_csv( 'output/crossval/best_inits_v12_30bins.csv')